In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np

In [32]:
filename = 'anomalousTrafficTest.txt' #'normalTrafficTest.txt'

In [33]:
data = pd.read_csv('../data/'+filename,sep='\n\n',header=None,names=['sentence'],)

print(data)

C:\Users\debon\AppData\Local\Temp\ipykernel_19252\3741842360.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('../data/'+filename,sep='\n\n',header=None,names=['sentence'],)


                                                 sentence
0       GET http://localhost:8080/tienda1/publico/anad...
1       User-Agent: Mozilla/5.0 (compatible; Konqueror...
2                                        Pragma: no-cache
3                                 Cache-control: no-cache
4       Accept: text/xml,application/xml,application/x...
...                                                   ...
305641        Accept-Charset: utf-8, utf-8;q=0.5, *;q=0.5
305642                                Accept-Language: en
305643                               Host: localhost:8080
305644  Cookie: JSESSIONID=3CC12010CDA952F123240EBAD79...
305645                                  Connection: close

[305646 rows x 1 columns]


In [34]:
data.value_counts().head(20)

sentence                                                                                                   
Accept-Charset: utf-8, utf-8;q=0.5, *;q=0.5                                                                    25065
Cache-control: no-cache                                                                                        25065
User-Agent: Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.8 (like Gecko)                            25065
Pragma: no-cache                                                                                               25065
Connection: close                                                                                              25065
Accept-Encoding: x-gzip, x-deflate, gzip, deflate                                                              25065
Accept-Language: en                                                                                            25065
Accept: text/xml,application/xml,application/xhtml+xml,text/html;q=0.9,te

In [35]:
top_10_freq_sent = [x[0] for x in data.value_counts().head(10).index.tolist()]
print([x[:x.index(':')] for x in top_10_freq_sent])

['Accept-Charset', 'Cache-control', 'User-Agent', 'Pragma', 'Connection', 'Accept-Encoding', 'Accept-Language', 'Accept', 'Host', 'Content-Type']


In [36]:
sent_to_remv = ['Cookie: ','Content-Length: ','POST http','Accept-Charset', 'User-Agent', 'Accept-Language', 'Accept', 'Cache-control', 'Connection', 'Host', 'Accept-Encoding', 'Pragma', 'Content-Type']

for freq_item in sent_to_remv:
    data = data.drop(data[data['sentence'].str.startswith(freq_item)].index)

In [37]:
print(data.value_counts().head(40))
print(data.describe())

sentence                                                                                        
B2A=Vaciar+carrito                                                                                  451
GET http://localhost:8080/tienda1/publico/vaciar.jsp?B2A=Vaciar+carrito HTTP/1.1                    451
GET http://localhost:8080/tienda1/publico/entrar.jsp?errorMsgA=Credenciales+incorrectas HTTP/1.1    444
errorMsgA=Credenciales+incorrectas                                                                  444
idA=1                                                                                               229
GET http://localhost:8080/tienda1/publico/caracteristicas.jsp?idA=1 HTTP/1.1                        229
idA=2                                                                                               222
GET http://localhost:8080/tienda1/publico/caracteristicas.jsp?idA=2 HTTP/1.1                        222
PUT http://localhost:8080/tienda1/publico/anadir.jsp HTTP/1.1          

In [38]:
from urllib.parse import unquote
import re

REPLACE_BY_SPACE_RE = re.compile('[^0-9a-zA-Z _]')
#REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')

def extract_text(text):
    if text.startswith('GET http'):
        url = text.split()[1]
        if '?' in url:
            text = url[url.index('?')+1:].strip()
        else:
            text = ''
    text = unquote(text) 
    
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    return text

data['sentence'] =data['sentence'].apply(extract_text)

In [39]:
data.describe()

,sentence
count,25462
unique,7278
top,
freq,5479


In [40]:
data = data.drop(data[data['sentence'].str.strip(' \t\n\r') ==''].index)

In [41]:
data=data.reset_index().drop('index',axis=1)

In [42]:
data.describe()

,sentence
count,19983
unique,7277
top,b2a vaciar carrito
freq,902


In [43]:
data.to_csv('../data/processed/'+filename,index=None,header=None,)